# Domains

In [1]:
import pandas as pd
import ipywidgets as widgets

In [2]:
with open('domain_blacklist.txt') as f:
    domains_blacklist = [a.strip() for a in f.readlines()]

In [3]:
def clean_url(url):
    if url.startswith('*.'):
        return url[2:]
    if url.startswith('www'):
        return url[4:]
    if url.startswith('*'):
        return url[1:]
    return url
domains_blacklist = [clean_url(a) for a in domains_blacklist]

In [5]:
domains_ds = pd.read_csv('domains_drug_substance.csv')
#domains_ds = pd.read_csv('domains_drug_package.csv')
#domains_ds = pd.read_csv('domains_analytical_development.csv')

In [7]:
def test_blacklist(url):
    return any([s in url for s in domains_blacklist])

In [8]:
#domains_ds[domains_ds['domain'].apply(test_blacklist)]

In [9]:
domains_all = domains_ds['domain'].unique()
domains_blacklisted = [domain for domain in domains_all if test_blacklist(domain)]

In [30]:
w_domains_list = widgets.SelectMultiple(
    options=domains_all,
    value=domains_blacklisted,
    rows=15,
    description='Domains_DP',
    disabled=False
)
w_domains_blacklist = widgets.SelectMultiple(
    options=domains_blacklist,
    value=[],
    rows=17,
    description='Blacklist',
    disabled=False
)

In [11]:
def select_activities(Activity = ['DP', 'DS', 'AD']):
    filenames = {'DP': 'domains_drug_package.csv', 'AD': 'domains_analytical_development.csv', 'DS': 'domains_drug_substance.csv'}
    filename = filenames[Activity]
    domains_ds = pd.read_csv(filename)
    domains_all = domains_ds['domain'].unique()
    domains_blacklisted = [domain for domain in domains_all if test_blacklist(domain)]
    w_domains_list.options = domains_all
    w_domains_list.value=domains_blacklisted
    w_domains_list.description = Activity



In [35]:
# All widgets

w_select_activity = widgets.interactive(select_activities)

btn_add_to_blacklist = widgets.Button(
    description='Add to blacklist',
    disabled=False,
    button_style='warning', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Blacklist!',
    layout=widgets.Layout(width='80%', position='right'),
    icon='check' # (FontAwesome names without the `fa-` prefix)
)

btn_remove_from_blacklist = widgets.Button(
    description='Remove from blacklist',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Not blacklist!',
    layout=widgets.Layout(width='80%', position='right'),
    icon='ban' # (FontAwesome names without the `fa-` prefix)
)

def add_to_blacklist(*args):
    global domains_blacklist
    selected_domains = w_domains_list.value
    if len(selected_domains) > 0:
        urls = [clean_url(domain) for domain in selected_domains]
        domains_blacklist.extend(urls)
        domains_blacklist.sort()
        w_domains_blacklist.options = domains_blacklist
        
def remove_from_blacklist(*args):
    global domains_blacklist
    selected_domains_ = w_domains_blacklist.value
    if len(selected_domains_) > 0:
        domains_blacklist = [domain for domain in domains_blacklist if domain not in selected_domains_]
        w_domains_blacklist.options = domains_blacklist

btn_add_to_blacklist.on_click(add_to_blacklist)

btn_remove_from_blacklist.on_click(remove_from_blacklist)

rLayout = widgets.Layout(align_items='flex-end')

### Domains management
Here the list of the left whows all the domains for the specific category, and domains from the blacklist are selected.

Right list show the current blacklist.

'Add to blacklist' adds selected domain from the left list to the right.

'Removes from blacklist' removes :)

In [37]:
widgets.HBox([widgets.VBox([w_select_activity, w_domains_list,btn_add_to_blacklist], layout=rLayout), widgets.VBox([w_domains_blacklist, btn_remove_from_blacklist], layout=rLayout)])